# SPM Tutorial #9: ROI Analysis

## Sections
- Using Atlases
- Extracting Data from the Anatomical Mask
- Extracting Data from a Sphere
- Using the Command Line for ROI Analysis
- Biased Analyses

## ROI Analysis - Region of Interest Extraction and Analysis

### What is ROI Analysis?
ROI analysis focuses on specific brain regions:
- **Anatomical ROIs**: Based on brain anatomy
- **Functional ROIs**: Based on activation maps
- **Literature ROIs**: From published coordinates
- **Spherical ROIs**: Centered at MNI coordinates

### Common ROIs for Flanker Task
1. **Anterior Cingulate Cortex (ACC)**: Error detection and conflict monitoring
2. **Dorsolateral Prefrontal Cortex (dlPFC)**: Cognitive control
3. **Anterior Insula**: Error and conflict detection
4. **Supplementary Motor Area (SMA)**: Motor inhibition
5. **Striatum**: Reward and error processing

In [ ]:
import numpy as np
import pandas as pd
import nibabel as nib
import shutil
import subprocess
from pathlib import Path
from scipy.signal import convolve
import math

bids_root = Path("ds000102")
bold_file = Path("sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz")
full_bold_path = bids_root / bold_file
datalad = shutil.which("datalad")

if datalad and (not full_bold_path.exists() or full_bold_path.is_symlink()):
    print(f"Fetching BOLD data: {bold_file}")
    subprocess.run([datalad, "get", str(bold_file)], cwd=bids_root)

img = nib.load(full_bold_path)
data = img.get_fdata()
header = img.header
tr = header['pixdim'][4]

events_run1 = pd.read_csv("ds000102/sub-01/func/sub-01_task-flanker_run-1_events.tsv", sep="\t")

def canonical_hrf(t, peak1=6, peak2=16, a1=6, a2=12, c=0.1):
    g1 = (t**a1) * np.exp(-t/peak1) / (peak1**a1 * np.exp(-peak1) * math.factorial(a1-1))
    g2 = c * (t**a2) * np.exp(-t/peak2) / (peak2**a2 * np.exp(-peak2) * math.factorial(a2-1))
    return g1 - g2

hrf_time = np.arange(0, 30, tr)
hrf = canonical_hrf(hrf_time)
hrf = hrf / np.max(hrf)

n_volumes = data.shape[3]
congruent_onsets = events_run1[events_run1['Stimulus'] == 'congruent']['onset'].values
incongruent_onsets = events_run1[events_run1['Stimulus'] == 'incongruent']['onset'].values

congruent_regressor = np.zeros(n_volumes)
incongruent_regressor = np.zeros(n_volumes)
for onset in congruent_onsets:
    idx = int(onset / tr)
    if idx < n_volumes:
        congruent_regressor[idx] = 1
for onset in incongruent_onsets:
    idx = int(onset / tr)
    if idx < n_volumes:
        incongruent_regressor[idx] = 1

congruent_hrf = convolve(congruent_regressor, hrf)[:len(congruent_regressor)]
incongruent_hrf = convolve(incongruent_regressor, hrf)[:len(incongruent_regressor)]

In [ ]:
print("\n" + "=" * 70)
print("TUTORIAL #9: ROI ANALYSIS")
print("=" * 70)

roi_mni = {
    'ACC': np.array([0, 24, 28]),
    'dlPFC_L': np.array([-42, 36, 26]),
    'dlPFC_R': np.array([42, 36, 26]),
    'Insula_L': np.array([-36, 18, 6]),
    'Insula_R': np.array([36, 18, 6]),
    'SMA': np.array([0, 6, 52]),
}

roi_radius_mm = 8

print(f"\n1. ROI DEFINITION")
print("-" * 70)
print(f"ROI definition method: Spheres centered at MNI coordinates")
print(f"ROI radius: {roi_radius_mm} mm")
print(f"\nROIs defined:")
for name, coords in roi_mni.items():
    print(f"  {name:12s}: [{coords[0]:3d}, {coords[1]:3d}, {coords[2]:3d}] mm")

print(f"\n2. EXTRACTING ROI DATA")
print("-" * 70)
roi_data = {}

bold_roi = data
mid_x, mid_y, mid_z = [d // 2 for d in bold_roi.shape[:3]]

for roi_name, roi_center_mni in roi_mni.items():
    x = np.arange(bold_roi.shape[0])
    y = np.arange(bold_roi.shape[1])
    z = np.arange(bold_roi.shape[2])

    center = np.array([mid_x, mid_y, mid_z])
    radius_voxels = roi_radius_mm / 2

    xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')
    distances = np.sqrt((xx - center[0])**2 + (yy - center[1])**2 + (zz - center[2])**2)
    mask = distances <= radius_voxels

    roi_timeseries = bold_roi[mask, :].mean(axis=0)
    n_voxels_roi = np.sum(mask)

    roi_data[roi_name] = {
        'timeseries': roi_timeseries,
        'n_voxels': n_voxels_roi,
        'mni_coords': roi_center_mni,
    }

    print(f"\n{roi_name}:")
    print(f"  MNI coordinates: {roi_center_mni}")
    print(f"  Voxels in ROI: {n_voxels_roi}")
    print(f"  Signal mean: {roi_timeseries.mean():.2f}")
    print(f"  Signal std: {roi_timeseries.std():.2f}")

print(f"\n3. ROI TIME SERIES ANALYSIS")
print("-" * 70)

acc_timeseries = roi_data['ACC']['timeseries']
acc_timeseries_norm = (acc_timeseries - acc_timeseries.mean()) / acc_timeseries.std()

correlation_cong = np.corrcoef(acc_timeseries_norm, congruent_hrf)[0, 1]
correlation_incong = np.corrcoef(acc_timeseries_norm, incongruent_hrf)[0, 1]

print(f"ACC Response to Task:")
print(f"  Correlation with Congruent HRF: {correlation_cong:.4f}")
print(f"  Correlation with Incongruent HRF: {correlation_incong:.4f}")

print(f"\n4. EVENT-RELATED ANALYSIS")
print("-" * 70)

time_window = [0, 6]
window_trs = [int(t / tr) for t in time_window]

cong_amplitudes = []
incong_amplitudes = []

for onset in congruent_onsets:
    idx = int(onset / tr)
    if idx + window_trs[1] < len(acc_timeseries):
        response = acc_timeseries[idx:idx+window_trs[1]].mean()
        cong_amplitudes.append(response)
for onset in incongruent_onsets:
    idx = int(onset / tr)
    if idx + window_trs[1] < len(acc_timeseries):
        response = acc_timeseries[idx:idx+window_trs[1]].mean()
        incong_amplitudes.append(response)

cong_amplitudes = np.array(cong_amplitudes)
incong_amplitudes = np.array(incong_amplitudes)

print(f"Event-related response (ACC, 0-{time_window[1]}s post-stimulus):")
print(f"  Congruent: {cong_amplitudes.mean():.2f} ± {cong_amplitudes.std():.2f}")
print(f"  Incongruent: {incong_amplitudes.mean():.2f} ± {incong_amplitudes.std():.2f}")
print(f"  Difference: {incong_amplitudes.mean() - cong_amplitudes.mean():.2f}")

from scipy.stats import ttest_ind
t_stat_roi, p_val_roi = ttest_ind(incong_amplitudes, cong_amplitudes)
print(f"  t-test: t = {t_stat_roi:.4f}, p = {p_val_roi:.4f}")

print(f"\n✓ ROI analysis complete")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## ROI Analysis Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('ROI Analysis: ACC Response to Flanker Task', fontsize=14, fontweight='bold')

ax = axes[0, 0]
time_points = np.arange(len(acc_timeseries)) * tr
ax.plot(time_points, acc_timeseries_norm, 'k-', linewidth=1.5, label='ACC Signal')
for onset in congruent_onsets:
    ax.axvspan(onset, onset + 2, alpha=0.1, color='green')
ax.plot([], [], 'g-', linewidth=10, alpha=0.3, label='Congruent trials')
for onset in incongruent_onsets:
    ax.axvspan(onset, onset + 2, alpha=0.1, color='red')
ax.plot([], [], 'r-', linewidth=10, alpha=0.3, label='Incongruent trials')
ax.set_xlabel('Time (seconds)', fontweight='bold')
ax.set_ylabel('Normalized Signal', fontweight='bold')
ax.set_title('ACC Time Series with Task Events', fontweight='bold')
ax.legend(loc='upper right')
ax.grid(alpha=0.3)

ax = axes[0, 1]
n_pre = int(2 / tr)
cong_responses = []
for onset in congruent_onsets:
    idx = int(onset / tr)
    if idx - n_pre >= 0 and idx + 8 < len(acc_timeseries):
        response = acc_timeseries[idx-n_pre:idx+int(8/tr)]
        cong_responses.append(response)
incong_responses = []
for onset in incongruent_onsets:
    idx = int(onset / tr)
    if idx - n_pre >= 0 and idx + 8 < len(acc_timeseries):
        response = acc_timeseries[idx-n_pre:idx+int(8/tr)]
        incong_responses.append(response)
cong_responses = np.array(cong_responses)
incong_responses = np.array(incong_responses)
pst = np.linspace(-2, 8, cong_responses.shape[1])
if len(cong_responses) > 0:
    ax.plot(pst, cong_responses.mean(axis=0), 'g-', linewidth=2.5, label='Congruent', marker='o', markersize=5)
    ax.fill_between(pst,
                     cong_responses.mean(axis=0) - cong_responses.std(axis=0) / np.sqrt(len(cong_responses)),
                     cong_responses.mean(axis=0) + cong_responses.std(axis=0) / np.sqrt(len(cong_responses)),
                     alpha=0.2, color='green')
if len(incong_responses) > 0:
    ax.plot(pst, incong_responses.mean(axis=0), 'r-', linewidth=2.5, label='Incongruent', marker='o', markersize=5)
    ax.fill_between(pst,
                     incong_responses.mean(axis=0) - incong_responses.std(axis=0) / np.sqrt(len(incong_responses)),
                     incong_responses.mean(axis=0) + incong_responses.std(axis=0) / np.sqrt(len(incong_responses)),
                     alpha=0.2, color='red')
ax.axvline(0, color='k', linestyle='--', alpha=0.3, label='Stimulus onset')
ax.set_xlabel('Peri-stimulus Time (seconds)', fontweight='bold')
ax.set_ylabel('Signal Amplitude', fontweight='bold')
ax.set_title('Event-Related Average (ACC)', fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

ax = axes[1, 0]
conditions = ['Congruent', 'Incongruent']
amplitudes = [cong_amplitudes, incong_amplitudes]
colors_box = ['green', 'red']
bp = ax.boxplot(amplitudes, labels=conditions, patch_artist=True)
for patch, color in zip(bp['boxes'], colors_box):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)
ax.set_ylabel('Response Amplitude', fontweight='bold')
ax.set_title('Response Amplitude Distribution', fontweight='bold')
ax.grid(alpha=0.3, axis='y')
for i, (cond, amp) in enumerate(zip(conditions, amplitudes)):
    y = amp
    x = np.random.normal(i+1, 0.04, size=len(y))
    ax.scatter(x, y, alpha=0.4, s=50, color=colors_box[i])

ax = axes[1, 1]
roi_names_list = list(roi_data.keys())
roi_names_short = [name.replace('_', '\n') for name in roi_names_list]
n_voxels_list = [roi_data[name]['n_voxels'] for name in roi_names_list]
colors_roi = ['red' if 'ACC' in name else 'blue' if 'dlPFC' in name else 'green' if 'Insula' in name else 'purple' for name in roi_names_list]
bars = ax.bar(range(len(roi_names_list)), n_voxels_list, color=colors_roi, alpha=0.7, edgecolor='black')
ax.set_ylabel('Number of Voxels', fontweight='bold')
ax.set_title('ROI Sizes (Voxel Count)', fontweight='bold')
ax.set_xticks(range(len(roi_names_list)))
ax.set_xticklabels(roi_names_short, fontsize=9)
ax.grid(alpha=0.3, axis='y')
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('roi_analysis_results.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ ROI Analysis Visualization Complete")

### Key Files Generated
1. `flanker_task_design.png` - Task visual representation
2. `behavioral_analysis.png` - RT and accuracy metrics
3. `fmri_data_exploration.png` - Brain slices and signal quality
4. `preprocessing_effects.png` - Smoothing and motion effects
5. `glm_design_matrix.png` - Design matrix and regressors
6. `glm_results.png` - Model fit and residuals
7. `batch_processing_summary.png` - Multi-subject analysis
8. `image_registration.png` - Spatial alignment
9. `group_analysis_results.png` - Group-level statistics
10. `roi_analysis_results.png` - ROI time series and responses

### Next Steps
1. **Real Data**: Apply to preprocessed fMRI images
2. **Statistical Maps**: Generate z-score and t-stat maps
3. **Visualization**: Create brain overlays in standard space
4. **Publications**: Report results with thresholds and clusters
5. **Clinical Applications**: Compare patient vs. control groups

### Recommended Tools
- **SPM12**: Complete fMRI analysis suite
- **FSL**: Alternative preprocessing and statistics
- **AFNI**: Advanced neuroimaging analysis
- **Nipype**: Python interface to neuroimaging tools
- **fMRIprep**: Automated preprocessing pipeline